In [7]:
# This is to rename the files into running number for easier keeping track.

import os
%matplotlib auto
from pathlib import Path
x=0
photo_dir=Path.cwd() /'dogpic' /'dataaugmentation/'
for i in os.listdir(photo_dir):
    my_dest='dogbatch'+str(x)+'.jpg'
    my_source=str(photo_dir)+'/'+i
    my_dest=str(photo_dir)+'/'+my_dest
    os.rename(my_source,my_dest)
    x+=1

Using matplotlib backend: MacOSX


In [6]:
#This is to utilize the imgaug libraries to augment the images
#imgaug is a library for image augmentaion in machine learning experiments. It supports a wide range of techniques.

#To download the imgaug library: https://imgaug.readthedocs.io/en/latest/source/installation.html


import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import cv2


from pathlib import Path
import os

ia.seed(1)


photo_dir=Path.cwd() /'dogpic' /'dataaugmentation/'
#print(photo_dir)
#imageio.imread('/Users/macbookpro/Downloads/dogpic/dataaugmentation/dogbatch0.jpg')


#The sequence of image augmentation is randomly selected.
#Images do not neccessarily undergo all augmentation steps. 
#Augmentation sequence applies affine transformations to images, flips some images horizontally,
#adds abit of blur and noise and also changes the contrast and brightness. 

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    
    ### cropping is not done here but later in the pipeline.config. 
    
    #iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.01))
    ),
    
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.1*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    
    iaa.Affine(rotate=(-25, 25)),
#     iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)})
    
    iaa.Affine(
       scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
       translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
       # rotate=(-25, 25),
        shear=(-8, 8),
        mode='symmetric'
    )
], random_order=True) # apply augmenters in random order

for i in os.listdir(photo_dir):
    i=Path.cwd() /'dogpic' /'dataaugmentation/'/i
    
    try:
        image=imageio.imread(i,pilmode='RGB')
        #ia.imshow(image)
        images_aug=seq(image=image)
        #images_aug=imageio.imread(images_aug,pilmode='RGB')
        #ia.imshow(images_aug)
        imageio.imwrite(i,images_aug)
    except:
        print(i)



/Users/macbookpro/Downloads/dogpic/dataaugmentation/d18.jpg
/Users/macbookpro/Downloads/dogpic/dataaugmentation/d8.jpg


/Users/macbookpro/opt/anaconda3/lib/python3.7/site-packages/imgaug/imgaug.py:2119: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(w, h), dpi=dpi)


/Users/macbookpro/Downloads/dogpic/dataaugmentation/d10.jpg
